# Setup

Prepare the statistical tests

In [20]:
from dataclasses import dataclass
from typing import Sequence

import numpy as np
from scipy.stats import shapiro, ttest_1samp, ttest_ind  # t

In [21]:
ALTERNATIVES = ("two-sided", "less", "greater")
ALPHA = 0.05

In [22]:
@dataclass
class Sample:
    values: Sequence[float]
    array: np.ndarray = None
    ddof: int = 1
    mean: float = None
    std: float = None
    variance: float = None

    def __post_init__(self):
        self.array = np.array(self.values)
        self.mean, self.std, self.var = self.get_basic_stats()

    def get_basic_stats(self) -> tuple[float, float, float]:
        mean = np.mean(self.array)
        std = np.std(self.array, ddof=self.ddof)
        var = np.var(self.array, ddof=self.ddof)
        return mean, std, var

    def __len__(self):
        return len(self.array)

    def __repr__(self):
        return f"Sample({self.values}"

    def __str__(self):
        return f"{self.array})"

In [23]:
# This was/is only a test whether my two_sample_t_test function works (it does)
def compare_two_samples(s1: Sample, s2: Sample) -> tuple[float, float]:
    t_stat, p_value = ttest_ind(s1.array, s2.array)  # just to check; fake!
    return t_stat, p_value

In [24]:
# Same calculation for both one and two sample t-tests
def _calculate_p_value(t_statistic: float, df: int, alternative: str) -> float:
    alternative = alternative.strip().lower()
    if alternative == "two_sided":
        p_value = (1 - t.cdf(abs(t_statistic), df)) * 2.0
    elif alternative == "less":
        p_value = t.cdf(t_statistic, df)
    elif alternative == "greater":
        p_value = 1 - t.cdf(t_statistic, df)
    else:
        raise ValueError("alternative must be 'two_sided', 'greater' or 'smaller'")

    return p_value

In [25]:
def is_normally_distributed(sample: Sample, alpha: float = ALPHA):
    _, p_value = shapiro(sample.array)
    return p_value > alpha

In [26]:
def one_sample_t_test(
    sample: Sample, population_mean: float, alternative: str = "two_sided"
) -> tuple[float, float]:
    # n = len(sample)
    # df = n - 1
    # t_statistic = (sample.mean - population_mean) / (sample.std / np.sqrt(n))
    # p_value = _calculate_p_value(t_statistic, df, alternative)

    if not is_normally_distributed(sample):
        print("Warning: sample is not normally distributed!")
        # print("Therefore, we do a Wilcoxon signed-rank test instead of a Student's t-test!")
        # test_statistic, p_value = wilcoxon(sample.array - population_mean, alternative=alternative)
        # return test_statistic, p_value
    test_statistic, p_value = ttest_1samp(
        sample.array, population_mean, alternative=alternative
    )
    return test_statistic, p_value

In [27]:
def two_sample_t_test(
    sample: Sample, baseline: Sample, alternative: str = "different"
) -> tuple[float, float]:
    # n1 = len(sample)
    # n2 = len(baseline)
    if sample.std > 2.0 * baseline.std or baseline.std > 2.0 * sample.std:
        print("Warning: standard deviations differ by more than a factor of 2")
        print("Therefore, we do a Welch's t-test instead of a Student's t-test!")
        equal_var = False
    else:
        equal_var = True

    # pooled_std = np.sqrt(((n1 - 1) * sample.std ** 2 + (n2 - 1) * baseline.std ** 2) / (n1 + n2 - 2))
    # t_statistic = (sample.mean - baseline.mean) / (pooled_std * np.sqrt(1 / n1 + 1 / n2))
    # df = n1 + n2 - 2
    # p_value = _calculate_p_value(t_statistic, df, alternative)

    if not is_normally_distributed(sample):
        print("Warning: sample is not normally distributed!")
        # print("Therefore, we do a Wilcoxon signed-rank test instead of a Student's t-test!")
        # test_statistic, p_value = wilcoxon(sample.array - population_mean, alternative=alternative)
        # return test_statistic, p_value
    if not is_normally_distributed(baseline):
        print("Warning: baseline is not normally distributed!")
        # print("Therefore, we do a Wilcoxon signed-rank test instead of a Student's t-test!")
        # test_statistic, p_value = wilcoxon(sample.array - population_mean, alternative=alternative)
        # return test_statistic, p_value

    test_statistic, p_value = ttest_ind(
        sample.array, baseline.array, equal_var=equal_var, alternative=alternative
    )
    return test_statistic, p_value

In [28]:
def interpret_p_value(
    p_value: float,
    # alternative: str,
    alpha: float = ALPHA,
) -> str:
    if p_value < alpha:
        return "Reject H0"
    else:
        return "Cannot reject H0"

# Example Usage

In [29]:
print("Shapiro-Wilk test example")
sample = Sample([2.5, 3.1, 2.8, 3.4, 2.9, 3.0, 3.3, 2.6, 3.2, 3.1])
result = is_normally_distributed(sample)
print(f"Sample is normally distributed: {result}")

Shapiro-Wilk test example
Sample is normally distributed: True


In [30]:
print("One sample t-test example")
sample = Sample([10.0, 11.0, 12.0, 13.0, 14.0])
population_mean = 14.0
print(f"Sample: {sample}")
print(f"Population mean: {population_mean}")
for alternative in ALTERNATIVES:
    t_stat, p_val = one_sample_t_test(sample, population_mean, alternative)
    print(
        f"Alternative: {alternative:9s}, "
        f"t-statistic: {t_stat:.3f}, "
        f"p-value: {p_val:.3f}, "
        f"Reject H0 (p<0.05): {p_val < 0.05}"
    )

print("\nTwo sample t-test example")
sample = Sample(np.array([10.0, 11.0, 12.0]))
baseline = Sample(np.array([12.0, 13.0, 14.0]))
print(f"Sample: {sample}")
print(f"Baseline: {baseline}")
for alternative in ALTERNATIVES:
    t_stat, p_val = two_sample_t_test(sample, baseline, alternative)
    print(
        f"Alternative: {alternative:9s}, "
        f"t-statistic: {t_stat:.3f}, "
        f"p-value: {p_val:.3f}, "
        f"Reject H0 (p<0.05): {p_val < 0.05}"
    )

One sample t-test example
Sample: [10. 11. 12. 13. 14.])
Population mean: 14.0
Alternative: two-sided, t-statistic: -2.828, p-value: 0.047, Reject H0 (p<0.05): True
Alternative: less     , t-statistic: -2.828, p-value: 0.024, Reject H0 (p<0.05): True
Alternative: greater  , t-statistic: -2.828, p-value: 0.976, Reject H0 (p<0.05): False

Two sample t-test example
Sample: [10. 11. 12.])
Baseline: [12. 13. 14.])
Alternative: two-sided, t-statistic: -2.449, p-value: 0.070, Reject H0 (p<0.05): False
Alternative: less     , t-statistic: -2.449, p-value: 0.035, Reject H0 (p<0.05): True
Alternative: greater  , t-statistic: -2.449, p-value: 0.965, Reject H0 (p<0.05): False


# Experiments

## Tokenizers

### Guacamol/SMILES

#### The initial "best" tokenizer

This is based on a single run for each tokenizer.
A single run consists of:

- a training with fixed seed
- a generation with fixed seed
- FCD as the metric 


#### Statistical tests for the "best" tokenizer

This is based on 5 runs for the "best" tokenizer.
The 5 runs are:

- a training with random seed
- a generation with fixed seed
- FCD as the metric

At this point we have 5 FCD values for the "best" tokenizer.
We then do a one-sample t-test and compare the sample with the "next best" tokenizer. If the current "best" tokenizer can be considered "done" and we declare the "best" tokenizer as the "winner". If not, we repeat the process with the "next best" tokenizer and perform a two-sample t-test. We repeat this process until we have a winner, or, we conculude that we have a couple of tokenizers which seem to perform equally well.


In [31]:
guacamol_char_wordpiece_176 = Sample(
    [
        0.21138082989205031,
        0.22424112983600253,
        0.22342369151375863,
        0.21396691344180852,
        0.22290331624026294,
    ]
)

guacamol_others: dict[str, float] = {
    "char_wordlevel_38": 0.22573631123455584,
    "char_bpe_44": 0.23582004914318588,
    "char_bpe_88": 0.22138152297816305,
    "char_bpe_176": 0.22446600524058624,
    "char_wordpiece_88": 0.24258628303761043,
    "char_unigram_44": 0.22958462926293066,
    "char_unigram_88": 0.2189468387861666,
    "atom_wordlevel_50": 0.23930838874149174,
    "smarts_wordlevel_106": 0.24132050338971567,
}

print(
    f"Initial best tokenizer char_wordpiece_176 has a mean of {guacamol_char_wordpiece_176.mean:.4f} and a std.dev. of {guacamol_char_wordpiece_176.std:.4f}"
)
print("\nCompare with single FCD value of other tokenizers (descending by FCD)")
alternative = "less"
print(f"Perform statistical test... (alternative: {alternative})")

for tokenizer, fcd in sorted(guacamol_others.items(), key=lambda x: x[1], reverse=True):
    print(f"\n{tokenizer:20s}: {fcd:.4f}")
    t_stat, p_val = one_sample_t_test(guacamol_char_wordpiece_176, fcd, alternative)
    print(
        f"t-statistic: {t_stat:.3f}, "
        f"p-value: {p_val:.3f}, "
        f"Reject H0 (p<0.05): {p_val < 0.05}"
    )

Initial best tokenizer char_wordpiece_176 has a mean of 0.2192 and a std.dev. of 0.0060

Compare with single FCD value of other tokenizers (descending by FCD)
Perform statistical test... (alternative: less)

char_wordpiece_88   : 0.2426
t-statistic: -8.677, p-value: 0.000, Reject H0 (p<0.05): True

smarts_wordlevel_106: 0.2413
t-statistic: -8.208, p-value: 0.001, Reject H0 (p<0.05): True

atom_wordlevel_50   : 0.2393
t-statistic: -7.462, p-value: 0.001, Reject H0 (p<0.05): True

char_bpe_44         : 0.2358
t-statistic: -6.168, p-value: 0.002, Reject H0 (p<0.05): True

char_unigram_44     : 0.2296
t-statistic: -3.856, p-value: 0.009, Reject H0 (p<0.05): True

char_wordlevel_38   : 0.2257
t-statistic: -2.430, p-value: 0.036, Reject H0 (p<0.05): True

char_bpe_176        : 0.2245
t-statistic: -1.959, p-value: 0.061, Reject H0 (p<0.05): False

char_bpe_88         : 0.2214
t-statistic: -0.815, p-value: 0.230, Reject H0 (p<0.05): False

char_unigram_88     : 0.2189
t-statistic: 0.088, p-val

## Results for the initial "best" tokenizer

The initial "best" tokenizer is char_wordpiece_176. After 5 runs and 5 FCD values we see, that the mean of those 5 values is worse than the single char_unigram_88 value. Therefore, we can expect that the char_wordpiece_176 tokenizer is not better than char_unigram_88. Also, its better value is not statistically significant comparing it to char_bpe_88 and char_bpe_176.

We move on with getting 5 FCD values of the 2nd best tokenizer, i.e. char_unigram_88 and performing a two-sample t-test of char_unigram_88 and char_wordpiece_176.


In [32]:
guacamol_char_unigram_88 = Sample(
    [
        0.22215763836744884,
        0.21625658903371914,
        0.21653028109176375,
        0.22646688628489642,
        0.23060648678460893,
    ]
)

guacamol_others: dict[str, float] = {
    "char_wordlevel_38": 0.22573631123455584,
    "char_bpe_44": 0.23582004914318588,
    "char_bpe_88": 0.22138152297816305,
    "char_bpe_176": 0.22446600524058624,
    "char_wordpiece_88": 0.24258628303761043,
    "char_unigram_44": 0.22958462926293066,
    "atom_wordlevel_50": 0.23930838874149174,
    "smarts_wordlevel_106": 0.24132050338971567,
}

print(
    f"Initial 2nd best tokenizer char_unigram_88 has a mean of {guacamol_char_unigram_88.mean:.4f} and a std.dev. of {guacamol_char_unigram_88.std:.4f}"
)
print("\nCompare with sample of initial best tokenizer (char_wordpiece_176)")
alternative = "less"
print(f"Perform statistical test... (alternative: {alternative})")
t_stat, p_val = two_sample_t_test(
    guacamol_char_unigram_88, guacamol_char_wordpiece_176, alternative
)
print(
    f"t-statistic: {t_stat:.3f}, "
    f"p-value: {p_val:.3f}, "
    f"Reject H0 (p<0.05): {p_val < 0.05}"
)


print("\nCompare with single FCD value of other tokenizers (descending by FCD)")
alternative = "less"
print(f"Perform statistical test... (alternative: {alternative})")

for tokenizer, fcd in sorted(guacamol_others.items(), key=lambda x: x[1], reverse=True):
    print(f"\n{tokenizer:20s}: {fcd:.4f}")
    t_stat, p_val = one_sample_t_test(guacamol_char_wordpiece_176, fcd, alternative)
    print(
        f"t-statistic: {t_stat:.3f}, "
        f"p-value: {p_val:.3f}, "
        f"Reject H0 (p<0.05): {p_val < 0.05}"
    )

Initial 2nd best tokenizer char_unigram_88 has a mean of 0.2224 and a std.dev. of 0.0062

Compare with sample of initial best tokenizer (char_wordpiece_176)
Perform statistical test... (alternative: less)
t-statistic: 0.829, p-value: 0.784, Reject H0 (p<0.05): False

Compare with single FCD value of other tokenizers (descending by FCD)
Perform statistical test... (alternative: less)

char_wordpiece_88   : 0.2426
t-statistic: -8.677, p-value: 0.000, Reject H0 (p<0.05): True

smarts_wordlevel_106: 0.2413
t-statistic: -8.208, p-value: 0.001, Reject H0 (p<0.05): True

atom_wordlevel_50   : 0.2393
t-statistic: -7.462, p-value: 0.001, Reject H0 (p<0.05): True

char_bpe_44         : 0.2358
t-statistic: -6.168, p-value: 0.002, Reject H0 (p<0.05): True

char_unigram_44     : 0.2296
t-statistic: -3.856, p-value: 0.009, Reject H0 (p<0.05): True

char_wordlevel_38   : 0.2257
t-statistic: -2.430, p-value: 0.036, Reject H0 (p<0.05): True

char_bpe_176        : 0.2245
t-statistic: -1.959, p-value: 0.

# Results for the 2nd “best” tokenizer

After getting 5 FCD values of the 2nd best tokenizer, i.e. char_unigram_88 and performing a two-sample t-test of char_unigram_88 and char_wordpiece_176 we can not rejct H0, i.e. the two tokenizers are not statistically different. Therefore, we can conclude that we have a couple of tokenizers which seem to perform equally well.

For the comparison with GuacaMol we select the model with a char_wordpiece_176 tokenizer with the lowest FCD value. The model directory is checkpoints/guacamol/tokenizers/char_wordpiece_176/2023-10-31_05-26-52_experiment.

We generate 5 samples of 10,000 molecules, this time with a random seed for each sample. We then calculate the FCD for each sample and compare the 5 samples with the value of GuacaMol with a single-sample one-sided t-test.

In [33]:
guacamol_char_wordpiece_176 = Sample(
    [
        0.2286251555231189,
        0.23513992637839465,
        0.21683749015160458,
        0.2195920751401701,
        0.22596241125135919,
    ]
)

guacamol_original = 0.455  # TODO replace with correct value

print(
    f"Selected tokenizer char_wordpiece_176 has a mean of {guacamol_char_wordpiece_176.mean:.4f} and a std.dev. of {guacamol_char_wordpiece_176.std:.4f}"
)
print("\nCompare with single FCD value of GuacaMol paper")
alternative = "less"
print(f"Perform statistical test... (alternative: {alternative})")

t_stat, p_val = one_sample_t_test(
    guacamol_char_wordpiece_176, guacamol_original, alternative
)
print(
    f"t-statistic: {t_stat:.3f}, "
    f"p-value: {p_val:.3f}, "
    f"Reject H0 (p<0.05): {p_val < 0.05}"
)

Selected tokenizer char_wordpiece_176 has a mean of 0.2252 and a std.dev. of 0.0073

Compare with single FCD value of GuacaMol paper
Perform statistical test... (alternative: less)
t-statistic: -70.491, p-value: 0.000, Reject H0 (p<0.05): True


In [34]:
# We then calculate mean and sigma of the other metrics

validity = Sample(
    [
        0.9779,
        0.9773,
        0.9753,
        0.9731,
        0.976,
    ]
)

uniqueness = Sample(
    [
        0.9989774005522037,
        0.9995907090964903,
        0.9994873372295704,
        0.9993834138320831,
        0.9995901639344262,
    ]
)

novelty = Sample(
    [
        0.9317227966014945,
        0.9375575801003173,
        0.9344480919162905,
        0.9374807197943444,
        0.9353218532185322,
    ]
)

fcd_g_mols = Sample(
    [
        0.955304605131761,
        0.9540606975651333,
        0.9575594241953026,
        0.9570320337197208,
        0.9558134869949808,
    ]
)

print("Other metrics of char_wordpiece_176:")
metrics: dict[str, Sample] = {
    "validity": validity,
    "uniqueness": uniqueness,
    "novelty": novelty,
    "fcd_g_mols": fcd_g_mols,
}

print("Other metrics of char_wordpiece_176:")
for name, sample in metrics.items():
    print(f"Metric: {name:12s} Mean: {sample.mean:.3f}   Std.dev. {sample.std:.3f}")

Other metrics of char_wordpiece_176:
Other metrics of char_wordpiece_176:
Metric: validity     Mean: 0.976   Std.dev. 0.002
Metric: uniqueness   Mean: 0.999   Std.dev. 0.000
Metric: novelty      Mean: 0.935   Std.dev. 0.002
Metric: fcd_g_mols   Mean: 0.956   Std.dev. 0.001


### USPTO50K/SMARTS

#### The initial "best" tokenizer

#### Statistical tests for the "best" tokenizer

In [39]:
uspto50k_char_wordlevel_47 = Sample(
    [
        728.0,
        701.0,
        702.0,
        703.0,
        710.0,
    ]
)

uspto50k_others: dict[str, float] = {
    "char_bpe_47": 721.0,
    "char_bpe_88": 696.0,
    "char_bpe_176": 618.0,
    "char_wordpiece_94": 587.0,
    "char_wordpiece_176": 557.0,
    "char_unigram_88": 621.0,
    "char_unigram_176": 616.0,
    "atom_wordlevel_86": 729.0,
    "smarts_wordlevel_947": 735.0,
}

print(
    f"Initial best tokenizer char_wordlevel_47 has a mean of {uspto50k_char_wordlevel_47.mean:.1f} and a std.dev. of {uspto50k_char_wordlevel_47.std:.1f}"
)
print("\nCompare with single 'known' value of other tokenizers (descending by 'known')")
alternative = "greater"
print(f"Perform statistical test... (alternative: {alternative})")

for tokenizer, known in sorted(
    uspto50k_others.items(), key=lambda x: x[1], reverse=False
):
    print(f"\n{tokenizer:20s}: {known:.4f}")
    t_stat, p_val = one_sample_t_test(uspto50k_char_wordlevel_47, known, alternative)
    print(
        f"t-statistic: {t_stat:.3f}, "
        f"p-value: {p_val:.3f}, "
        f"Reject H0 (p<0.05): {p_val < 0.05}"
    )

Initial best tokenizer char_wordlevel_47 has a mean of 703.2 and a std.dev. of 4.0

Compare with single 'known' value of other tokenizers (descending by 'known')
Perform statistical test... (alternative: greater)

char_bpe_176        : 700.0000
t-statistic: 1.806, p-value: 0.073, Reject H0 (p<0.05): False

char_wordpiece_94   : 700.0000
t-statistic: 1.806, p-value: 0.073, Reject H0 (p<0.05): False

char_wordpiece_176  : 700.0000
t-statistic: 1.806, p-value: 0.073, Reject H0 (p<0.05): False

char_unigram_88     : 700.0000
t-statistic: 1.806, p-value: 0.073, Reject H0 (p<0.05): False

char_unigram_176    : 700.0000
t-statistic: 1.806, p-value: 0.073, Reject H0 (p<0.05): False

atom_wordlevel_86   : 700.0000
t-statistic: 1.806, p-value: 0.073, Reject H0 (p<0.05): False

smarts_wordlevel_947: 700.0000
t-statistic: 1.806, p-value: 0.073, Reject H0 (p<0.05): False

char_bpe_88         : 701.0000
t-statistic: 1.242, p-value: 0.141, Reject H0 (p<0.05): False

char_bpe_47         : 702.0000
t-s